### Importing libraries

In [25]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
sns.set_style('darkgrid')

from bokeh.layouts import row, column, widgetbox
from bokeh.models import CustomJS, Slider, CDSView, BooleanFilter
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook

### Reading data

In [26]:
df = pd.read_csv('data.csv', sep=',', dtype={'date': 'object', 'rating': 'object', 'ead': 'float'}, parse_dates=['date'])
df['date'] = [d.year for d in df['date']]
# df['date'] = df['date'].dt.date
buckets = df['rating'].unique()
g = df.groupby(['date'])['ead'].sum()
print(g)
df.iloc[0, :]
print(g.index.dtype)

date
2017    100.0
2018    100.0
2019    100.0
2020    100.0
2021    100.0
2022    100.0
Name: ead, dtype: float64
int64


### Barchart for a single year using view

In [27]:
year = 2017   # date = dt.datetime.strptime(year, '%d-%m-%Y').date()
booleans = df['date']==year
print(df[booleans])

source = ColumnDataSource(data=df)
plot = figure(x_range=buckets, plot_width=400, plot_height=400)

view = CDSView(source=source, filters=[BooleanFilter(booleans)])
plot.vbar(x='rating', top='ead', width=0.9, source=source, view=view)
output_notebook()
show(plot)

   date rating    ead
0  2017    R01    0.0
1  2017    R02  100.0
2  2017    R03    0.0
3  2017    R04    0.0
4  2017      D    0.0


Loading BokehJS ...

### Barchart for multiple years using a slider

In [28]:
# Starting position of the bar chart
booleans = df['date']==year
source = ColumnDataSource(data=df[booleans])
source_orig = ColumnDataSource(data=df)

plot = figure(x_range=buckets, y_range=(0, 100), plot_width=400, plot_height=400)

plot.vbar(x='rating', top='ead', width=0.9, source=source)

# Create callback -> TODO: make it nicer?
callback = CustomJS(args=dict(source=source, source_orig=source_orig), code="""
    source.data['rating'] = []
    source.data['ead'] = []
    for (var i = 0; i < source_orig.data['date'].length; i++) {
        if (source_orig.data['date'][i]==slider.value) {
            source.data['rating'].push(source_orig.data['rating'][i])
            source.data['ead'].push(source_orig.data['ead'][i])  
        }
    }
    source.change.emit();
""")

# Add the slider
start = int(df['date'].min())
end = int(df['date'].max())

year_slider = Slider(start=start, end=end, value=year, step=1, title="Year", callback=callback)
callback.args["slider"] = year_slider

# Create layout
layout = column(
    widgetbox(year_slider),
    plot,
)

output_notebook()
show(layout)

Loading BokehJS ...